# Imports and base class

In [ ]:
%%writefile submission.py

import pandas as pd
import numpy as np
import json
import random
import operator
from collections import defaultdict
import collections
import math


# base class for all agents, random agent
class agent():
    def initial_step(self):
        return np.random.randint(3)
    
    def history_step(self, history):
        return np.random.randint(3)
    
    def step(self, history):
        if len(history) == 0:
            return int(self.initial_step())
        else:
            return int(self.history_step(history)) 

# simple agents from Larchenko

In [ ]:
%%writefile -a submission.py
        
# anti random cracking
class anti_random_cracker(agent):
    def initial_step(self):
        random.seed(np.random.random())
        return np.random.randint(3)
    
    def history_step(self, history):
        random.seed(np.random.random())
        return np.random.randint(3)
    
        
# agent that returns (previousCompetitorStep + shift) % 3
class mirror_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['competitorStep'] + self.shift) % 3
    
    
# agent that returns (previousPlayerStep + shift) % 3
class self_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['step'] + self.shift) % 3    


# agent that beats the most popular step of competitor
class popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['competitorStep'] for x in history])
        return (int(np.argmax(counts)) + 1) % 3

    
# agent that beats the agent that beats the most popular step of competitor
class anti_popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['step'] for x in history])
        return (int(np.argmax(counts)) + 2) % 3

# Agents from rps contest
## rank 1 IO2_fightinguuuu

In [ ]:
%%writefile -a submission.py

# IO2_fightinguuuu
# http://www.rpscontest.com/entry/885001
class IO2(agent):
    def __init__(self, mode='normal'):
        num_predictor = 27
        self.my_his = ''
        self.your_his = ''
        self.both_his = ''
        self.list_predictor = [''] * num_predictor
        self.length = 0
        self.score_predictor = [0]*num_predictor
        self.predictors = []
        
        self.mode = mode
        
    def history_step(self, history):
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        num_predictor = 27

        len_rfind = [20]
        limit = [10, 20, 60]
        beat = {'R': 'P', 'P': 'S', 'S': 'R'}
        not_lose = {'R':'PPR', 'P':'SSP', 'S':'RRS'}
        
        temp1 = { "PP":"1" , "PR":"2" , "PS":"3",
                  "RP":"4" , "RR":"5", "RS":"6",
                  "SP":"7" , "SR":"8", "SS":"9"}
        temp2 = { "1":"PP","2":"PR","3":"PS",
                    "4":"RP","5":"RR","6":"RS",
                    "7":"SP","8":"SR","9":"SS"} 
        who_win = { "PP": 0, "PR":1 , "PS":-1,
                    "RP": -1,"RR":0, "RS":1,
                    "SP": 1, "SR":-1, "SS":0}

        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
            
        if len(history) == 1:
            self.predictors = [output] * num_predictor


        # 2 different lengths of history, 3 kinds of history, both, mine, yours
        # 3 different limit length of reverse learning
        # 6 kinds of strategy based on Iocaine Powder
        #update predictors
        #"""
        if len(self.list_predictor[0])<5:
            front =0
        else:
            front =1
        for i in range (num_predictor):
            if self.predictors[i]==input:
                result ="1"
            else:
                result ="0"
            self.list_predictor[i] = self.list_predictor[i][front:5]+result #only 5 rounds before
        #history matching 1-6
        self.my_his += output
        self.your_his += input
        self.both_his += temp1[input+output]
        self.length +=1
        for i in range(1):
            len_size = min(self.length,len_rfind[i])
            j=len_size
            #both_his
            while j>=1 and not self.both_his[self.length-j:self.length] in self.both_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.both_his.rfind(self.both_his[self.length-j:self.length],0,self.length-1)
                self.predictors[0+6*i] = self.your_his[j+k]
                self.predictors[1+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[0+6*i] = random.choice("RPS")
                self.predictors[1+6*i] = random.choice("RPS")
            j=len_size
            #your_his
            while j>=1 and not self.your_his[self.length-j:self.length] in self.your_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.your_his.rfind(self.your_his[self.length-j:self.length],0,self.length-1)
                self.predictors[2+6*i] = self.your_his[j+k]
                self.predictors[3+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[2+6*i] = random.choice("RPS")
                self.predictors[3+6*i] = random.choice("RPS")
            j=len_size
            #my_his
            while j>=1 and not self.my_his[self.length-j:self.length] in self.my_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.my_his.rfind(self.my_his[self.length-j:self.length],0,self.length-1)
                self.predictors[4+6*i] = self.your_his[j+k]
                self.predictors[5+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[4+6*i] = random.choice("RPS")
                self.predictors[5+6*i] = random.choice("RPS")

        for i in range(3):
            temp =""
            search = temp1[(output+input)] #last round
            for start in range(2, min(limit[i],self.length) ):
                if search == self.both_his[self.length-start]:
                    temp+=self.both_his[self.length-start+1]
            if(temp==""):
                self.predictors[6+i] = random.choice("RPS")
            else:
                collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
                for sdf in temp:
                    next_move = temp2[sdf]
                    if(who_win[next_move]==-1):
                        collectR[temp2[sdf][1]]+=3
                    elif(who_win[next_move]==0):
                        collectR[temp2[sdf][1]]+=1
                    elif(who_win[next_move]==1):
                        collectR[beat[temp2[sdf][0]]]+=1
                max1 = -1
                p1 =""
                for key in collectR:
                    if(collectR[key]>max1):
                        max1 = collectR[key]
                        p1 += key
                self.predictors[6+i] = random.choice(p1)

        #rotate 9-27:
        for i in range(9,27):
            self.predictors[i] = beat[beat[self.predictors[i-9]]]

        #choose a predictor
        len_his = len(self.list_predictor[0])
        for i in range(num_predictor):
            S = 0
            for j in range(len_his):
                if self.list_predictor[i][j]=="1":
                    S+=(j+1)*(j+1)
                else:
                    S-=(j+1)*(j+1)
            self.score_predictor[i] = S

        max_score = max(self.score_predictor)
        #min_score = min(score_predictor)
        #c_temp = {"R":0,"P":0,"S":0}
        #for i in range (num_predictor):
            #if score_predictor[i]==max_score:
            #    c_temp[predictors[i]] +=1
            #if score_predictor[i]==min_score:
            #    c_temp[predictors[i]] -=1
        if max_score>0:
            predict = self.predictors[self.score_predictor.index(max_score)]
        else:
            predict = random.choice(self.your_his)
        
        output = random.choice(not_lose[predict])

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 2 dllu1

In [ ]:
%%writefile -a submission.py

# dllu1
# http://www.rpscontest.com/entry/498002
class dllu1(agent):
    def __init__(self, mode='normal'):
        numPre = 30
        numMeta = 6
        self.moves=['','','','']
        self.pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
        self.soma = [0,0,0,0,0,0,0,0,0];
        self.best = [0,0,0];
        self.length=0
        self.mScore = None
        self.p = None
        self.m = None
        
        self.mode = mode
        
    def initial_step(self):
        
        numPre = 30
        numMeta = 6
        beat={'R':'P','P':'S','S':'R'}
        self.p=[random.choice("RPS")]*numPre
        self.m=[random.choice("RPS")]*numMeta
        self.mScore=[5,2,5,2,4,2]
        self.rps = [1,1,1];
        
        rev_dic = {'R':0, 'P':1, 'S':2}
        
        output = beat[self.m[self.mScore.index(max(self.mScore))]]
        if max(self.mScore)<0.07 or random.randint(3,40)>self.length:
            output=beat[random.choice("RPS")]
            
        return rev_dic[output]
        
    def history_step(self, history):
        numPre = 30
        numMeta = 6

        limit = 8
        beat={'R':'P','P':'S','S':'R'}
        
        centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
        centripete={'R':0,'P':1,'S':2}
        
        
        a="RPS"
     
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R':0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        for i in range(numPre):
            pp = self.p[i]
            bpp = beat[pp]
            bbpp = beat[bpp]
            self.pScore[0][i]=0.9*self.pScore[0][i]+((input==pp)-(input==bbpp))*3
            self.pScore[1][i]=0.9*self.pScore[1][i]+((output==pp)-(output==bbpp))*3
            self.pScore[2][i]=0.87*self.pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
            self.pScore[3][i]=0.87*self.pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
            self.pScore[4][i]=(self.pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
            self.pScore[5][i]=(self.pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
        for i in range(numMeta):
            self.mScore[i]=0.96*(self.mScore[i]+(input==self.m[i])-(input==beat[beat[self.m[i]]]))
        self.soma[centrifuge[input+output]] +=1;
        self.rps[centripete[input]] +=1;
        self.moves[0]+=str(centrifuge[input+output])
        self.moves[1]+=input
        self.moves[2]+=output
        self.length+=1
        for y in range(3):
            j=min([self.length,limit])
            while j>=1 and not self.moves[y][self.length-j:self.length] in self.moves[y][0:self.length-1]:
                j-=1
            i = self.moves[y].rfind(self.moves[y][self.length-j:self.length],0,self.length-1)
            self.p[0+2*y] = self.moves[1][j+i] 
            self.p[1+2*y] = beat[self.moves[2][j+i]]
        j=min([self.length,limit])
        while j>=2 and not self.moves[0][self.length-j:self.length-1] in self.moves[0][0:self.length-2]:
            j-=1
        i = self.moves[0].rfind(self.moves[0][self.length-j:self.length-1],0,self.length-2)
        if j+i>=self.length:
            self.p[6] = self.p[7] = random.choice("RPS")
        else:
            self.p[6] = self.moves[1][j+i] 
            self.p[7] = beat[self.moves[2][j+i]]

        self.best[0] = self.soma[centrifuge[output+'R']]*self.rps[0]/self.rps[centripete[output]]
        self.best[1] = self.soma[centrifuge[output+'P']]*self.rps[1]/self.rps[centripete[output]]
        self.best[2] = self.soma[centrifuge[output+'S']]*self.rps[2]/self.rps[centripete[output]]
        self.p[8] = self.p[9] = a[self.best.index(max(self.best))]

        for i in range(10,numPre):
            self.p[i]=beat[beat[self.p[i-10]]]

        for i in range(0,numMeta,2):
            self.m[i]=       self.p[self.pScore[i  ].index(max(self.pScore[i  ]))]
            self.m[i+1]=beat[self.p[self.pScore[i+1].index(max(self.pScore[i+1]))]]
            
        output = beat[self.m[self.mScore.index(max(self.mScore))]]
        if max(self.mScore)<0.07 or random.randint(3,40)>self.length:
            output=beat[random.choice("RPS")]

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 3 RPS_Meta_Fix

In [ ]:
%%writefile -a submission.py
    
# RPS_Meta_Fix
# http://www.rpscontest.com/entry/5649874456412160
class RPS_Meta_Fix(agent):
    
    def __init__(self, mode='normal'):
        self.DNA=[""]*3
        self.prin=[random.choice("RPS")]*18
        self.meta=[random.choice("RPS")]*6
        self.skor1=[[0]*18,[0]*18,[0]*18,[0]*18,[0]*18,[0]*18]
        self.skor2=[0]*6
        
        self.mode = mode
        
    def initiatl_step(self):
        RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
        mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
        rot={'R':'P','P':'S','S':'R'}
        
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        output=rot[self.meta[self.skor2.index(max(self.skor2))]]
        return rev_dic[output]
        
        
    def history_step(self, history):
        RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
        mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        R, P, S = 0, 1, 2
        index = {"R": R, "P": P, "S": S}
        beat = ("P", "S", "R")
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        rot={'R':'P','P':'S','S':'R'}
        for j in range(18):
            for i in range(4):
                self.skor1[i][j]*=0.8
            for i in range(4,6):
                self.skor1[i][j]*=0.5
            for i in range(0,6,2):
                self.skor1[i][j]-=(input==rot[rot[self.prin[j]]])
                self.skor1[i+1][j]-=(output==rot[rot[self.prin[j]]])
            for i in range(2,6,2):
                self.skor1[i][j]+=(input==self.prin[j])
                self.skor1[i+1][j]+=(output==self.prin[j])
                self.skor1[0][j]+=1.3*(input==self.prin[j])-0.3*(input==rot[self.prin[j]])
                self.skor1[1][j]+=1.3*(output==self.prin[j])-0.3*(output==rot[self.prin[j]])
        for i in range(6):
            self.skor2[i]=0.9*self.skor2[i]+(input==self.meta[i])-(input==rot[rot[self.meta[i]]])
        self.DNA[0]+=input
        self.DNA[1]+=output
        self.DNA[2]+=RNA[input+output]
        for i in range(3):
            j=min(21,len(self.DNA[2]))
            k=-1
            while j>1 and k<0:
                j-=1
                k=self.DNA[i].rfind(self.DNA[i][-j:],0,-1)
            self.prin[2*i]=self.DNA[0][j+k]
            self.prin[2*i+1]=rot[self.DNA[1][j+k]]
            k=self.DNA[i].rfind(self.DNA[i][-j:],0,j+k-1)
            self.prin[2*i]=mix[self.prin[2*i]+self.DNA[0][j+k]]
            self.prin[2*i+1]=mix[self.prin[2*i+1]+rot[self.DNA[1][j+k]]]
        for i in range(6,18):
            self.prin[i]=rot[self.prin[i-6]]
        for i in range(0,6,2):
            self.meta[i]=self.prin[self.skor1[i].index(max(self.skor1[i]))]
            self.meta[i+1]=rot[self.prin[self.skor1[i+1].index(max(self.skor1[i+1]))]]
            
        output=rot[self.meta[self.skor2.index(max(self.skor2))]]
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3
    

## rank 4 meta-sgd-markov-full-ew

In [ ]:
%%writefile -a submission.py

# meta-sgd-markov-full-ew
# http://www.rpscontest.com/entry/5728994095792128
class meta_sgd_markov_full_ew(agent):
    
    
    def __init__(self, mode='normal'):

        self.USE_RANDOM = 1
        self.USE_MARKOW = 1

        self.LAST_ROUND = 1000
        self.ROUND = 1

        self.history = []
        self.moves = ["R","P","S"]
        self.beatedBy = {"R":"P", "P":"S", "S":"R"}
        self.result = {"R":{"R":0, "P":-1, "S":1}, "P":{"R":1, "P":0, "S":-1}, "S":{"R":-1, "P":1, "S":0}}

        self.alpha = 0.01
        self.M = 0

        if self.USE_RANDOM == 1:
            self.M += 1

        if self.USE_MARKOW == 1:
            self.markov_orders = [0,1,2,3,4,5,6]
            self.historyCount = {}
            self.M += 6 * len(self.markov_orders)


        self.weight = [1] * self.M
        self.decay = [0.85] * self.M

        self.score = [0] * self.M
        self.selected = [0] * self.M
        self.move = [random.choice(self.moves) for i in range(self.M)]
        
        self.mode = mode
        
    def initial_step(self):
        def selectBest(s):
            return random.choice([i for i in range(len(s)) if max(s) == s[i]])
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        best = selectBest(self.score)
        self.selected[best] += 1
        output = self.move[best]
        self.last = output
        
        return rev_dic[output]
        
    def history_step(self, history):
        
        def selectBest(s):
            return random.choice([i for i in range(len(s)) if max(s) == s[i]])

        def selectBestDict(s):

            ew = {i:s[self.beatedBy[self.beatedBy[i]]] - s[self.beatedBy[i]] for i in s.keys()};
            return random.choice([i for i in ew.keys() if max(ew.values()) == ew[i]])
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        self.ROUND += 1
        self.history += [(self.last,input)]
        self.score = [ self.decay[i] * self.score[i] + self.weight[i] * self.result[self.move[i]][input] for i in range(self.M)]
        self.weight = [ self.weight[i] + self.alpha * self.result[self.move[i]][input] for i in range(self.M)]

        index = 0
        # random optimal
        if self.USE_RANDOM == 1:
            self.move[index] = random.choice(self.moves)
            # adjust random optimal score to zero
            self.score[index] = 0
            index += 1

        first_meta_index = index

        if self.USE_MARKOW == 1:
            # markow with meta strategies
            for m in self.markov_orders:
                if len(self.history) > m:
                    key = tuple(self.history[-m-1:-1])
                    if not (key in self.historyCount):
                        self.historyCount[key] = [{"R":0,"P":0,"S":0},{"R":0,"P":0,"S":0}]
                    self.historyCount[key][0][self.history[-1][0]] += 1
                    self.historyCount[key][1][self.history[-1][1]] += 1

            for m in self.markov_orders:
                if len(self.history) >= m:
                    key = tuple(self.history[-m:])
                    if key in self.historyCount:
                        self.move[index]   = selectBestDict(self.historyCount[key][0])
                        self.move[index+3] = selectBestDict(self.historyCount[key][1])
                    else:
                        self.move[index]   = random.choice(self.moves)
                        self.move[index+3] = random.choice(self.moves)
                else:
                    self.move[index]   = random.choice(self.moves)
                    self.move[index+3] = random.choice(self.moves)
                index += 6

        # set other meta strategies
        for i in range(first_meta_index, self.M, 3):
            self.move[i+1] = self.beatedBy[self.move[i]]
            self.move[i+2] = self.beatedBy[self.move[i+1]]
            
        best = selectBest(self.score)
        self.selected[best] += 1
        output = self.move[best]
        self.last = output
        
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 5 Are you a lucker?

In [ ]:
%%writefile -a submission.py

class AreYouALucker(agent):
    def __init__(self, mode='normal'):
        self.mode = mode
        
    def initial_step(self):
        
        num_predictors =27
        num_meta= 18

        len_rfind = [20]
        limit = [10,20,60]
        
        self.your_his =""
        self.my_his = ""
        self.both_his=""
        self.both_his2=""
        self.length =0
        self.score1=[3]*num_predictors
        self.score2=[3]*num_predictors
        self.score3=[3]*num_predictors
        self.score4=[3]*num_predictors
        self.score5=[3]*num_predictors
        self.score6=[3]*num_predictors
        self.metascore=[3]*num_meta
        
        self.chance =[0]*num_predictors
        self.chance2 =[0]*num_predictors
        output = random.choice("RPS")
        self.predictors = [output]*num_predictors
        self.metapredictors = [output]*num_meta
        
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
    
    def history_step(self, history):
        
        num_predictors =27
        num_meta= 18

        len_rfind = [20]
        limit = [10,20,60]
        
        beat = { "P":"S" , "R":"P" , "S":"R" }
        not_lose = { "R":"PR", "P":"SP", "S":"RS" } 
        
        temp1 = { "PP":"1","PR":"2","PS":"3",
                  "RP":"4","RR":"5","RS":"6",
                  "SP":"7","SR":"8","SS":"9"}
        temp2 = { "1":"PP","2":"PR","3":"PS",
                    "4":"RP","5":"RR","6":"RS",
                    "7":"SP","8":"SR","9":"SS"} 
        who_win = { "PP": 0, "PR":1 , "PS":-1,
                    "RP": -1,"RR":0, "RS":1,
                    "SP": 1, "SR":-1, "SS":0}
        index = { "P":0, "R":1, "S":2 }
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        
        #calculate score
        for i in range(num_predictors):
            #meta 1
            self.score1[i]*=0.8
            if input==self.predictors[i]:
                self.score1[i]+=3
            else:
                self.score1[i]-=3
            #meta 2
            if input==self.predictors[i]:
                self.score2[i]+=3
            else:
                self.score2[i]=0
            #meta 3
            self.score3[i]*=0.8
            if output==self.predictors[i]:
                self.score3[i]+=3
            else:
                self.score3[i]-=3
            #meta 4
            if output==self.predictors[i]:
                self.score4[i]+=3
            else:
                self.score4[i]=0
            #meta 5
            self.score5[i]*=0.8
            if input==self.predictors[i]:
                self.score5[i]+=3
            else:
                if self.chance[i]==1:
                    self.chance[i]=0
                    self.score5[i]-=3
                else:
                    self.chance[i]=1
                    self.score5[i]=0
            #meta 6
            self.score6[i]*=0.8
            if output==self.predictors[i]:
                self.score6[i]+=3
            else:
                if self.chance2[i]==1:
                    self.chance2[i]=0
                    self.score6[i]-=3
                else:
                    self.chance2[i]=1
                    self.score6[i]=0
        #calculate metascore
        for i in range(num_meta):
            self.metascore[i]*=0.9
            if input==self.metapredictors[i]:
                self.metascore[i]+=3
            else:
                self.metascore[i]=0
        #Predictors
        #if length>1:
        #    output=beat[predict]
        self.your_his+=input
        self.my_his+=output
        self.both_his+=temp1[(input+output)]
        self.both_his2+=temp1[(output+input)]
        self.length+=1

        #history matching
        for i in range(1):
            len_size = min(self.length,len_rfind[i])
            j=len_size
            #both_his
            while j>=1 and not self.both_his[self.length-j:self.length] in self.both_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.both_his.rfind(self.both_his[self.length-j:self.length],0,self.length-1)
                self.predictors[0+6*i] = self.your_his[j+k]
                self.predictors[1+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[0+6*i] = random.choice("RPS")
                self.predictors[1+6*i] = random.choice("RPS")
            j=len_size
            #your_his
            while j>=1 and not self.your_his[self.length-j:self.length] in self.your_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.your_his.rfind(self.your_his[self.length-j:self.length],0,self.length-1)
                self.predictors[2+6*i] = self.your_his[j+k]
                self.predictors[3+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[2+6*i] = random.choice("RPS")
                self.predictors[3+6*i] = random.choice("RPS")
            j=len_size
            #my_his
            while j>=1 and not self.my_his[self.length-j:self.length] in self.my_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.my_his.rfind(self.my_his[self.length-j:self.length],0,self.length-1)
                self.predictors[4+6*i] = self.your_his[j+k]
                self.predictors[5+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[4+6*i] = random.choice("RPS")
                self.predictors[5+6*i] = random.choice("RPS")

        #Reverse
        for i in range(3):
            temp =""
            search = temp1[(output+input)] #last round
            for start in range(2, min(limit[i],self.length) ):
                if search == self.both_his2[self.length-start]:
                    temp+=self.both_his2[self.length-start+1]
            if(temp==""):
                self.predictors[6+i] = random.choice("RPS")
            else:
                collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
                for sdf in temp:
                    next_move = temp2[sdf]
                    if(who_win[next_move]==-1):
                        collectR[temp2[sdf][1]]+=3
                    elif(who_win[next_move]==0):
                        collectR[temp2[sdf][1]]+=1
                    elif(who_win[next_move]==1):
                        collectR[beat[temp2[sdf][0]]]+=1
                max1 = -1
                p1 =""
                for key in collectR:
                    if(collectR[key]>max1):
                        max1 = collectR[key]
                        p1 += key
                self.predictors[6+i] = random.choice(p1)

        for i in range(9,27):
            self.predictors[i]=beat[beat[self.predictors[i-9]]]
        #find prediction for each meta
        self.metapredictors[0]=self.predictors[self.score1.index(max(self.score1))]
        self.metapredictors[1]=self.predictors[self.score2.index(max(self.score2))]
        self.metapredictors[2]=beat[self.predictors[self.score3.index(max(self.score3))]]
        self.metapredictors[3]=beat[self.predictors[self.score4.index(max(self.score4))]]
        self.metapredictors[4]=self.predictors[self.score5.index(max(self.score5))]
        self.metapredictors[5]=beat[self.predictors[self.score6.index(max(self.score6))]]
        for i in range(6,18):
            self.metapredictors[i] = beat[self.metapredictors[i-6]]

        predict = self.metapredictors[self.metascore.index(max(self.metascore))]
        output = beat[predict]
        #output = random.choice(not_lose[predict])
    
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 6 RPS_FSa

In [ ]:
%%writefile -a submission.py

class RPS_FSa(agent):
    def __init__(self, mode='normal'):
        self.mode = mode
        
        self.DNA=""
        self.tRNA=""
        self.mRNA=""
        self.flag=[False]*7
        self.hist=[[0]*3]*4
        self.eval=[0]*3
        self.subs=[0]*30
        self.prin=[[0]*30]*3
        self.meta=[0]*3
   
    
    def initial_step(self):
        
        output=random.choice("RPS")
    
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
    
    def history_step(self, history):
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        kon={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
        d0s={'1':0,'2':0,'3':0,'4':1,'5':1,'6':1,'7':2,'8':2,'9':2}
        d1s={'1':0,'2':1,'3':2,'4':0,'5':1,'6':2,'7':0,'8':1,'9':2}
        k2i={'R':0,'P':1,'S':2}
        i2k={0:'R',1:'P',2:'S'}

        def mdl(N):
            N%=3
            if N<0:
                N+=3
            return N

        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
            
        for i in range(3):
            j=self.prin[i].index(max(self.prin[i]))
            if ((j<3 and self.flag[0]) or (3<=j<6 and self.flag[1]) or (6<=j<9 and self.flag[2]) or (9<=j<12 and self.flag[3]) or
           (12<=j<18 and self.flag[4]) or (18<=j<24 and self.flag[5]) or (j>=24 and self.flag[6])):
                self.meta[i]*=random.betavariate(4.6,1.4)
                k=mdl(self.subs[j]-k2i[input])
                if k==2:
                    self.meta[i]-=1
                else:
                    self.meta[i]+=k
        for j in range(30):
            if ((j<3 and self.flag[0]) or (3<=j<6 and self.flag[1]) or (6<=j<9 and self.flag[2]) or (9<=j<12 and self.flag[3]) or
           (12<=j<18 and self.flag[4]) or (18<=j<24 and self.flag[5]) or (j>=24 and self.flag[6])):
                for i in range(1,3):
                    self.prin[i][j]*=0.9
                k=mdl(self.subs[j]-k2i[input])
                if k==1:
                    for i in range(3):
                        self.prin[i][j]+=1
                elif k==2:
                    for i in range(3):
                        if i<2:
                            self.prin[i][j]-=1
                        elif self.prin[i][j]<3:
                            self.prin[i][j]-=1.5
                        else:
                            self.prin[i][j]*=0.5
                elif self.prin[2][j]<3:
                    self.prin[2][j]-=0.25
                else:
                    self.prin[2][j]=0.75*self.prin[2][j]+0.5
        self.DNA+=kon[input+output]
        self.tRNA+=input
        self.mRNA+=output
        for i in range(4):
            for j in range(3):
                self.hist[i][j]=0
        i=min(26,len(self.DNA))
        j=-1
        w=0
        while i>1 and j<0:
            i-=1
            RNA=self.DNA[-i:]
            j=self.DNA.find(RNA,0,-1)
        self.flag[4]=(j>=0)
        while j>=0:
            k=i+j
            w+=1
            self.hist[0][d0s[self.DNA[k]]]+=1
            self.hist[1][d1s[self.DNA[k]]]+=1
            self.hist[2][d0s[self.DNA[k]]]+=w
            self.hist[3][d1s[self.DNA[k]]]+=w
            j=self.DNA.find(RNA,j+1,-1)
        if self.flag[4]:
            j=d0s[self.DNA[k]]+1
            k=d1s[self.DNA[k]]-1
            for i in range(3):
                self.subs[i+12]=mdl(j-i)
                self.subs[i+15]=mdl(k-i)
        for i in range(4):
            self.flag[i]=(self.hist[i][0]!=self.hist[i][1] or self.hist[i][1]!=self.hist[i][2])
            if self.flag[i]:
                for j in range(3):
                    self.eval[j]=self.hist[i][mdl(j-1)]-self.hist[i][mdl(j+1)]
                k=self.eval.index(max(self.eval))
                for j in range(3):
                    self.subs[3*i+j]=mdl(k-j+i%2)
        i=min(26,len(self.DNA))
        j=-1
        while i>1 and j<0:
            i-=1
            RNA=self.tRNA[-i:]
            j=self.tRNA.rfind(RNA,0,-1)
        self.flag[5]=(j>=0)
        if self.flag[5]:
            k=k2i[self.mRNA[i+j]]-1
            j=k2i[self.tRNA[i+j]]+1
            for i in range(3):
                self.subs[i+18]=mdl(j-i)
                self.subs[i+21]=mdl(k-i)
        i=min(26,len(self.DNA))
        j=-1
        while i>1 and j<0:
            i-=1
            RNA=self.mRNA[-i:]
            j=self.mRNA.rfind(RNA,0,-1)
        self.flag[6]=(j>=0)
        if self.flag[6]:
            k=k2i[self.mRNA[i+j]]-1
            j=k2i[self.tRNA[i+j]]+1
            for i in range(3):
                self.subs[i+24]=mdl(j-i)
                self.subs[i+27]=mdl(k-i)
        i=self.meta.index(max(self.meta))
        j=self.prin[i].index(max(self.prin[i]))
        if ((j<3 and self.flag[0]) or (3<=j<6 and self.flag[1]) or (6<=j<9 and self.flag[2]) or (9<=j<12 and self.flag[3]) or
        (12<=j<18 and self.flag[4]) or (18<=j<24 and self.flag[5]) or (j>=24 and self.flag[6])):
            output=i2k[self.subs[j]]
        else:
            output=random.choice("RPS")

    
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 8 SkipTree6/6

In [ ]:
%%writefile -a submission.py
    
# SkipTree6/6
# http://www.rpscontest.com/entry/5152734005166080
class SkipTree6_6(agent):
    
    def __init__(self, mode='normal'):
        self.mode = mode


    class MarkovTree:
        def __init__(self, counts = None):
            self.counts = [0 for _ in range(3)]
            self.children = None
            self.total = 0

        def update_helper(self, h, i, p, d, skips):
            stop = False
            for j in range(p, len(h)):
                k = h[j]
                self.counts[i] += 2
                self.total += 2
                if stop or d >= 6:
                    return
                d += 1
                if self.children is None:
                    self.children = [None for _ in range(4)]
                    self.children[3] = SkipTree6_6.MarkovTree()
                if self.children[k] is None:
                    self.children[k] = SkipTree6_6.MarkovTree()
                    stop = True
                self.children[3].update_helper(h, i, j + 1, d, skips + 1)
                self = self.children[k]

        def update(self, h, i):
            self.update_helper(h, i, 0, 0, 0)

        def predict_helper(self, h, p, n0):
            for j in range(p, len(h)):
                k = h[j]
                for i, x in enumerate(self.counts):
                    n0[i] += x
                if self.children is None:
                    return
                self.children[3].predict_helper(h, j + 1, n0)
                child = self.children[k]
                if child is None:
                    return
                self = child

        def predict(self, h):
            n0 = [1, 1, 1]
            self.predict_helper(h, 0, n0)
            return n0


    def initial_step(self):
        rev_dic = {'R': 0, 'P':1, 'S':2}
        beat = ("P", "S", "R")
        self.tree = self.MarkovTree()
        self.history = collections.deque([])
        
        counts = self.tree.predict(self.history)
        t = sum(counts)
        r = random.uniform(0, t)
        x = 0
        for i, p in enumerate(counts):
            x += p
            if r <= x:
                break
        output = beat[i]
        return rev_dic[output]
        

    def history_step(self, history):
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        R, P, S = 0, 1, 2
        index = {"R": R, "P": P, "S": S}
        beat = ("P", "S", "R")
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        i = index[input]
        j = index[output]

        self.tree.update(self.history, i)
        self.history.appendleft(i)
        self.history.appendleft(j)
        

        counts = self.tree.predict(self.history)
        t = sum(counts)
        r = random.uniform(0, t)
        x = 0
        for i, p in enumerate(counts):
            x += p
            if r <= x:
                break

        output = beat[i]
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3
    

## rank 9 UCB10

In [ ]:
%%writefile -a submission.py
    
# UCB10
# http://www.rpscontest.com/entry/5723151296102400
class UCB10(agent):
    
    def __init__(self, mode='normal'):
        self.mode = mode
    
    class MarkovTree:
        
        def __init__(self, parent = None):
            self.counts = [0.0 for _ in range(3)]
            self.visits = [0.0 for _ in range(3)]
            self.children = None
            self.parent = parent
            
            self.gamma = random.gammavariate
            self.sqrt = math.sqrt
            self.log = math.log
            

        def score(self, t):
            def ucb(s, n, t):
                return s + self.sqrt(2 * self.log(t) / n)

            return ucb(self.pos_total, self.neg_total, self.total_visits, t)

        def select_move(self, t):
            def ucb(s, n, t):
                return s + self.sqrt(2 * self.log(t) / n)
            def belief(xs):
                n = sum(xs)
                m = sum(1 for x in xs if x)
                if m == 0:
                    a = 1.0
                else:
                    a = m / (6.0 * self.log((n + 1.0) / m))
                return [self.gamma(x + a, 1) for x in xs]

            r, p, s = belief(self.counts)
            visits = belief(self.visits)
            scores = [s - p, r - s, p - r]
            scores = [ucb(s, k, t) for s, k in zip(scores, visits)]
            best = max(scores)
            return (best, scores.index(best))

        def update(self, h, i):
            for n in h:
                self.counts[i] += 1
                if self.children is None:
                    break
                self = self.children[n]
                if self is None:
                    break

        def predict(self, h):
            stop = False
            t = 1
            for d, n in enumerate(h):
                t += sum(self.visits)
                if stop or d >= 16:
                    break
                if self.children is None:
                    self.children = [None for _ in range(3)]
                if self.children[n] is None:
                    self.children[n] = UCB10.MarkovTree(self)
                    stop = True
                child = self.children[n]
                self = child
            best_score = float("-inf")
            leaf = self
            while self is not None:
                score, move = self.select_move(t)
                if score >= best_score:
                    best_score = score
                    best_move = move
                self = self.parent
            return (leaf, best_move)
        
        
    def initial_step(self):
        rev_dic = {'R': 0, 'P':1, 'S':2}
        name = ("R", "P", "S")
        self.tree = self.MarkovTree()
        self.history = collections.deque([])
        self.node = self.tree
        
        self.node, k = self.tree.predict(self.history)
        output = name[k]
        
        return rev_dic[output]
    
    def history_step(self, history):
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        R, P, S = 0, 1, 2
        index = {"R": R, "P": P, "S": S}
        beat = (P, S, R)
        name = ("R", "P", "S")
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        i = index[input]
        j = index[output]

        while self.node is not None:
            self.node.visits[j] += 1
            self.node = self.node.parent
        self.tree.update(self.history, i)

        self.history.appendleft(i)
        self.history.appendleft(j)

        self.node, k = self.tree.predict(self.history)
        output = name[k]
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 10 Testing Please Ignore

In [ ]:
%%writefile -a submission.py

# Testing Please Ignore
# http://www.rpscontest.com/entry/342001
class Testing_Please_Ignore(agent):
    
    def __init__(self, mode='normal'):
        self.mode = mode
    

    def initial_step(self):
        rps = ['R', 'P', 'S']
        
        self.played_probs = defaultdict(lambda: 1)
        self.dna_probs = [defaultdict(lambda: defaultdict(lambda: 1)) for i in range(18)]

        self.wlt_probs = [defaultdict(lambda: 1) for i in range(9)]

        self.answers = [{'c': 1, 'b': 1, 'r': 1} for i in range(12)]

        self.patterndict = [defaultdict(str) for i in range(6)]

        self.consec_strat_usage = [[0]*6,[0]*6,[0]*6] #consecutive strategy usage
        self.consec_strat_candy = [[],   [],   []   ] #consecutive strategy candidates

        output = random.choice(rps)
        self.histories = ["","",""]
        self.dna = ["" for i in range(12)]

        self.sc = 0
        self.strats = [[] for i in range(3)] 
        
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]



    def history_step(self, history):
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        score  = {'RR': 0, 'PP': 0, 'SS': 0, \
                  'PR': 1, 'RS': 1, 'SP': 1, \
                  'RP': -1, 'SR': -1, 'PS': -1,}
        cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', \
                  'PR': 'b', 'RS': 'b', 'SP': 'b', \
                  'RP': 'c', 'SR': 'c', 'PS': 'c',}
        beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        rps = ['R', 'P', 'S']
        wlt = {1:0,-1:1,0:2}

        def counter_prob(probs):
            weighted_list = []
            for h in rps:
                weighted = 0
                for p in probs.keys():
                    points = score[h+p]
                    prob = probs[p]
                    weighted += points * prob
                weighted_list.append((h, weighted))

            return max(weighted_list, key=operator.itemgetter(1))[0]

        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        
        prev_sc = self.sc

        self.sc = score[output + input]
        for j in range(3):
            prev_strats = self.strats[j][:]
            for i, c in enumerate(self.consec_strat_candy[j]):
                if c == input:
                    self.consec_strat_usage[j][i] += 1
                else:
                    self.consec_strat_usage[j][i] = 0
            m = max(self.consec_strat_usage[j])
            self.strats[j] = [i for i, c in enumerate(self.consec_strat_candy[j]) if self.consec_strat_usage[j][i] == m]

            for s1 in prev_strats:
                for s2 in self.strats[j]:
                    self.wlt_probs[j*3+wlt[prev_sc]][chr(s1)+chr(s2)] += 1

            if self.dna[2*j+0] and self.dna[2*j+1]:
                self.answers[2*j+0][cscore[input+self.dna[2*j+0]]] += 1
                self.answers[2*j+1][cscore[input+self.dna[2*j+1]]] += 1
            if self.dna[2*j+6] and self.dna[2*j+7]:
                self.answers[2*j+6][cscore[input+self.dna[2*j+6]]] += 1
                self.answers[2*j+7][cscore[input+self.dna[2*j+7]]] += 1

            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2*j][self.histories[j][-length:]]
                if pattern:
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        self.patterndict[2*j+1][pattern[-length2:]] += output + input
                self.patterndict[2*j][self.histories[j][-length:]] += output + input
        self.played_probs[input] += 1
        self.dna_probs[0][self.dna[0]][input] +=1
        self.dna_probs[1][self.dna[1]][input] +=1
        self.dna_probs[2][self.dna[1]+self.dna[0]][input] +=1
        self.dna_probs[9][self.dna[6]][input] +=1
        self.dna_probs[10][self.dna[6]][input] +=1
        self.dna_probs[11][self.dna[7]+self.dna[6]][input] +=1

        self.histories[0] += output + input
        self.histories[1] += input
        self.histories[2] += output

        self.dna = ["" for i in range(12)]
        for j in range(3):
            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2*j][self.histories[j][-length:]]
                if pattern != "":
                    self.dna[2*j+1] = pattern[-2]
                    self.dna[2*j+0]  = pattern[-1]
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        pattern2 = self.patterndict[2*j+1][pattern[-length2:]]
                        if pattern2 != "":
                            self.dna[2*j+7] = pattern2[-2]
                            self.dna[2*j+6] = pattern2[-1]
                            break
                    break

        probs = {}
        for hand in rps:
            probs[hand] = self.played_probs[hand]

        for j in range(3):
            if self.dna[j*2] and self.dna[j*2+1]:
                for hand in rps:
                    probs[hand] *= self.dna_probs[j*3+0][self.dna[j*2+0]][hand] * \
                                   self.dna_probs[j*3+1][self.dna[j*2+1]][hand] * \
                                   self.dna_probs[j*3+2][self.dna[j*2+1]+self.dna[j*2+0]][hand]
                    probs[hand] *= self.answers[j*2+0][cscore[hand+self.dna[j*2+0]]] * \
                                   self.answers[j*2+1][cscore[hand+self.dna[j*2+1]]]
                self.consec_strat_candy[j] = [self.dna[j*2+0], beat[self.dna[j*2+0]], cede[self.dna[j*2+0]],\
                                         self.dna[j*2+1], beat[self.dna[j*2+1]], cede[self.dna[j*2+1]]]
                strats_for_hand = {'R': [], 'P': [], 'S': []}
                for i, c in enumerate(self.consec_strat_candy[j]):
                    strats_for_hand[c].append(i)
                pr = self.wlt_probs[wlt[self.sc]+3*j]
                for hand in rps:
                    for s1 in self.strats[j]:
                        for s2 in strats_for_hand[hand]:
                            probs[hand] *= pr[chr(s1)+chr(s2)]
            else:
                self.consec_strat_candy[j] = []
        for j in range(3):
            if self.dna[j*2+6] and self.dna[j*2+7]:
                for hand in rps:
                    probs[hand] *= self.dna_probs[j*3+9][self.dna[j*2+6]][hand] * \
                                   self.dna_probs[j*3+10][self.dna[j*2+7]][hand] * \
                                   self.dna_probs[j*3+11][self.dna[j*2+7]+self.dna[j*2+6]][hand]
                    probs[hand] *= self.answers[j*2+6][cscore[hand+self.dna[j*2+6]]] * \
                                   self.answers[j*2+7][cscore[hand+self.dna[j*2+7]]]

        output = counter_prob(probs)
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 15 mybot

In [ ]:
%%writefile -a submission.py
    
# mybot
# http://www.rpscontest.com/entry/5725657602457600
class mybot(agent):
    
    def __init__(self, mode='normal'):
        
        self.score  = {'RR': 0, 'PP': 0, 'SS': 0, \
                  'PR': 1, 'RS': 1, 'SP': 1, \
                  'RP': -1, 'SR': -1, 'PS': -1,}
        self.cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', \
                  'PR': 'b', 'RS': 'b', 'SP': 'b', \
                  'RP': 'c', 'SR': 'c', 'PS': 'c',}
        self.beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        self.cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        self.rps = ['R', 'P', 'S']
        self.wlt = {1:0,-1:1,0:2}
        
        self.mode = mode
        
    def initial_step(self):

        
        self.played_probs = defaultdict(lambda: 1)
        self.dna_probs = [defaultdict(lambda: defaultdict(lambda: 1)) for i in range(18)]

        self.wlt_probs = [defaultdict(lambda: 1) for i in range(9)]

        self.answers = [{'c': 1, 'b': 1, 'r': 1} for i in range(12)]

        self.patterndict = [defaultdict(str) for i in range(6)]

        self.consec_strat_usage = [[0]*6,[0]*6,[0]*6] #consecutive strategy usage
        self.consec_strat_candy = [[],   [],   []   ] #consecutive strategy candidates

        output = random.choice(self.rps)
        self.histories = ["","",""]
        self.dna = ["" for i in range(12)]

        self.sc = 0
        self.strats = [[] for i in range(3)] 
        
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
        
    def history_step(self, history):
        
        def counter_prob(probs):
            weighted_list = []
            for h in self.rps:
                weighted = 0
                for p in probs.keys():
                    points = self.score[h+p]
                    prob = probs[p]
                    weighted += points * prob
                weighted_list.append((h, weighted))

            return max(weighted_list, key=operator.itemgetter(1))[0]

        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        prev_sc = self.sc

        self.sc = self.score[output + input]
        for j in range(3):
            prev_strats = self.strats[j][:]
            for i, c in enumerate(self.consec_strat_candy[j]):
                if c == input:
                    self.consec_strat_usage[j][i] += 1
                else:
                    self.consec_strat_usage[j][i] = 0
            m = max(self.consec_strat_usage[j])
            self.strats[j] = [i for i, c in enumerate(self.consec_strat_candy[j]) if self.consec_strat_usage[j][i] == m]

            for s1 in prev_strats:
                for s2 in self.strats[j]:
                    self.wlt_probs[j*3+self.wlt[prev_sc]][chr(s1)+chr(s2)] += 1

            if self.dna[2*j+0] and self.dna[2*j+1]:
                self.answers[2*j+0][self.cscore[input+self.dna[2*j+0]]] += 1
                self.answers[2*j+1][self.cscore[input+self.dna[2*j+1]]] += 1	
            if self.dna[2*j+6] and self.dna[2*j+7]:
                self.answers[2*j+6][self.cscore[input+self.dna[2*j+6]]] += 1
                self.answers[2*j+7][self.cscore[input+self.dna[2*j+7]]] += 1

            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2*j][self.histories[j][length:]]
                if pattern:
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        self.patterndict[2*j+1][pattern[-length2:]] += output + input
                self.patterndict[2*j][self.histories[j][-length:]] += output + input
        self.played_probs[input] += 1
        self.dna_probs[0][self.dna[0]][input] +=1
        self.dna_probs[1][self.dna[1]][input] +=1
        self.dna_probs[2][self.dna[1]+self.dna[0]][input] +=1
        self.dna_probs[9][self.dna[6]][input] +=1
        self.dna_probs[10][self.dna[6]][input] +=1
        self.dna_probs[11][self.dna[7]+self.dna[6]][input] +=1

        self.histories[0] += output + input
        self.histories[1] += input
        self.histories[2] += output

        self.dna = ["" for i in range(12)]
        for j in range(3):
            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2*j][self.histories[j][-length:]]
                if pattern != "":
                    self.dna[2*j+1] = pattern[-2]
                    self.dna[2*j+0]  = pattern[-1]
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        pattern2 = self.patterndict[2*j+1][pattern[-length2:]]
                        if pattern2 != "":
                            self.dna[2*j+7] = pattern2[-2]
                            self.dna[2*j+6] = pattern2[-1]
                            break
                    break

        probs = {}
        for hand in self.rps:
            probs[hand] = self.played_probs[hand]

        for j in range(3):
            if self.dna[j*2] and self.dna[j*2+1]:
                for hand in self.rps:
                    probs[hand] *= self.dna_probs[j*3+0][self.dna[j*2+0]][hand] * \
                                   self.dna_probs[j*3+1][self.dna[j*2+1]][hand] * \
                                   self.dna_probs[j*3+2][self.dna[j*2+1]+self.dna[j*2+0]][hand]
                    probs[hand] *= self.answers[j*2+0][self.cscore[hand+self.dna[j*2+0]]] * \
                                   self.answers[j*2+1][self.cscore[hand+self.dna[j*2+1]]]
                self.consec_strat_candy[j] = [self.dna[j*2+0], self.beat[self.dna[j*2+0]], self.cede[self.dna[j*2+0]],\
                                         self.dna[j*2+1], self.beat[self.dna[j*2+1]], self.cede[self.dna[j*2+1]]]
                strats_for_hand = {'R': [], 'P': [], 'S': []}
                for i, c in enumerate(self.consec_strat_candy[j]):
                    strats_for_hand[c].append(i)
                pr = self.wlt_probs[self.wlt[self.sc]+3*j]
                for hand in self.rps:
                    for s1 in self.strats[j]:
                        for s2 in strats_for_hand[hand]:
                            probs[hand] *= pr[chr(s1)+chr(s2)]
            else:
                self.consec_strat_candy[j] = []
        for j in range(3):
            if self.dna[j*2+6] and self.dna[j*2+7]:
                for hand in self.rps:
                    probs[hand] *= self.dna_probs[j*3+9][self.dna[j*2+6]][hand] * \
                                   self.dna_probs[j*3+10][self.dna[j*2+7]][hand] * \
                                   self.dna_probs[j*3+11][self.dna[j*2+7]+self.dna[j*2+6]][hand]
                    probs[hand] *= self.answers[j*2+6][self.cscore[hand+self.dna[j*2+6]]] * \
                                   self.answers[j*2+7][self.cscore[hand+self.dna[j*2+7]]]

        output = counter_prob(probs)
    

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 17 The Illiad

In [ ]:
%%writefile -a submission.py

class TheIlliad(agent):
    def __init__(self, mode='normal'):
        self.mode = mode
        
        numPre = 30
        numMeta = 6

        limit = 8
        beat={'R':'P','P':'S','S':'R'}
        self.moves=['','','','']
        self.pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
        centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
        centripete={'R':0,'P':1,'S':2}
        self.soma = [0,0,0,0,0,0,0,0,0];
        rps = [1,1,1];
        a="RPS"
        self.best = [0,0,0];
        self.length=0
        self.p=[random.choice("RPS")]*numPre
        self.m=[random.choice("RPS")]*numMeta
        self.mScore=[5,2,5,2,4,2]
        dithering = 0.7
    
    def initial_step(self):
        beat={'R':'P','P':'S','S':'R'}
        
        output = beat[self.m[self.mScore.index(max(self.mScore))]]
        if max(self.mScore)<3+random.random() or random.randint(3,40)>self.length or random.random() < 0.5:
            output=beat[random.choice("RPS")]
    
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
    
    def history_step(self, history):
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        numPre = 30
        numMeta = 6
        limit = 8
        beat={'R':'P','P':'S','S':'R'}
        centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
        centripete={'R':0,'P':1,'S':2}
        rps = [1,1,1];
        a="RPS"
        dithering = 0.7

        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
            
        for i in range(numPre):
            pp = self.p[i]
            bpp = beat[pp]
            bbpp = beat[bpp]
            self.pScore[0][i]=0.9*self.pScore[0][i]+((input==pp)-(input==bbpp))*3
            self.pScore[1][i]=0.9*self.pScore[1][i]+((output==pp)-(output==bbpp))*3
            self.pScore[2][i]=0.87*self.pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
            self.pScore[3][i]=0.87*self.pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
            self.pScore[4][i]=(self.pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
            self.pScore[5][i]=(self.pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
        for i in range(numMeta):
            self.mScore[i]=0.96*(self.mScore[i]+(input==self.m[i])-(input==beat[beat[self.m[i]]])) + (random.random()-0.5)*dithering
        self.soma[centrifuge[input+output]] +=1;
        rps[centripete[input]] +=1;
        self.moves[0]+=str(centrifuge[input+output])
        self.moves[1]+=input
        self.moves[2]+=output
        self.length+=1
        for y in range(3):
            j=min([self.length,limit])
            while j>=1 and not self.moves[y][self.length-j:self.length] in self.moves[y][0:self.length-1]:
                j-=1
            i = self.moves[y].rfind(self.moves[y][self.length-j:self.length],0,self.length-1)
            self.p[0+2*y] = self.moves[1][j+i] 
            self.p[1+2*y] = beat[self.moves[2][j+i]]
        j=min([self.length,limit])
        while j>=2 and not self.moves[0][self.length-j:self.length-1] in self.moves[0][0:self.length-2]:
            j-=1
        i = self.moves[0].rfind(self.moves[0][self.length-j:self.length-1],0,self.length-2)
        if j+i>=self.length:
            self.p[6] = self.p[7] = random.choice("RPS")
        else:
            self.p[6] = self.moves[1][j+i] 
            self.p[7] = beat[self.moves[2][j+i]]

        self.best[0] = self.soma[centrifuge[output+'R']]*rps[0]/rps[centripete[output]]
        self.best[1] = self.soma[centrifuge[output+'P']]*rps[1]/rps[centripete[output]]
        self.best[2] = self.soma[centrifuge[output+'S']]*rps[2]/rps[centripete[output]]
        self.p[8] = self.p[9] = a[self.best.index(max(self.best))]

        for i in range(10,numPre):
            self.p[i]=beat[beat[self.p[i-10]]]

        for i in range(0,numMeta,2):
            self.m[i]=       self.p[self.pScore[i  ].index(max(self.pScore[i  ]))]
            self.m[i+1]=beat[self.p[self.pScore[i+1].index(max(self.pScore[i+1]))]]
            
        output = beat[self.m[self.mScore.index(max(self.mScore))]]
        if max(self.mScore)<3+random.random() or random.randint(3,40)>self.length or random.random() < 0.5:
            output=beat[random.choice("RPS")]

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 36 RV2_fixing

In [ ]:
%%writefile -a submission.py

#RV2_fixing
# http://www.rpscontest.com/entry/5649874456412160
class RV2_fixing(agent):
    
    def __init__(self, mode='normal'):
        self.mode = mode

    def initial_step(self):
        
        num_predictors =45
        num_meta= 4
        
        self.your_his =""
        self.my_his = ""
        self.both_his=""
        self.both_his2=""
        self.length =0
        
        self.score1=[3]*num_predictors
        self.score2=[3]*num_predictors
        self.score3=[3]*num_predictors
        self.score4=[3]*num_predictors
        self.metascore=[3]*num_meta
        
        
        output = random.choice("RPS")
        self.predictors = [output]*num_predictors
        self.metapredictors = [output]*num_meta
    
        
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
        
    def history_step(self, history):
        
        num_predictors =45
        num_meta= 4
        
        len_rfind = [20,6]
        limit = [10,20,60]
        beat = { "P":"S" , "R":"P" , "S":"R" }
        not_lose = { "R":"PPPR", "P":"SSSP", "S":"RRRS" } #50-50 chance # 75% test with testing please ignore = 57.1% vs 41.2%, 50% = 55.2% vs 43.1%
        
        
        temp1 = { "PP":"1","PR":"2","PS":"3",
                  "RP":"4","RR":"5","RS":"6",
                  "SP":"7","SR":"8","SS":"9"}
        temp2 = { "1":"PP","2":"PR","3":"PS",
                    "4":"RP","5":"RR","6":"RS",
                    "7":"SP","8":"SR","9":"SS"} 
        who_win = { "PP": 0, "PR":1 , "PS":-1,
                    "RP": -1,"RR":0, "RS":1,
                    "SP": 1, "SR":-1, "SS":0}
        index = { "P":0, "R":1, "S":2 }
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        
        #calculate score
        for i in range(num_predictors):
            #meta 1
            self.score1[i]*=0.8
            if input==self.predictors[i]:
                self.score1[i]+=3
            elif input==beat[beat[self.predictors[i]]]:
                self.score1[i]-=3
            #meta 2
            if input==self.predictors[i]:
                self.score2[i]+=3
            elif input==beat[beat[self.predictors[i]]]:
                self.score2[i]=0
            #meta 3
            self.score3[i]*=0.8
            self.score3[i]+=(output==self.predictors[i])*3
            self.score3[i]-=(output==beat[beat[self.predictors[i]]])*3
            #meta 4
            self.score4[i]+=(output==self.predictors[i])*3
            if output==beat[beat[self.predictors[i]]]:
                    self.score4[i]=0
        #calculate metascore
        for i in range(num_meta):
            self.metascore[i]*=0.95
            if input==self.metapredictors[i]:
                self.metascore[i]+=3
            elif input==beat[beat[self.predictors[i]]]:
                self.metascore[i]-=3
        #Predictors
        self.your_his+=input
        self.my_his+=output
        self.both_his+=temp1[(input+output)]
        self.both_his2+=temp1[(output+input)]
        self.length+=1

        #history matching
        for i in range(2):
            len_size = min(self.length,len_rfind[i])
            j=len_size
            #both_his
            while j>=1 and not self.both_his[self.length-j:self.length] in self.both_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.both_his.rfind(self.both_his[self.length-j:self.length],0,self.length-1)
                self.predictors[0+6*i] = self.your_his[j+k]
                self.predictors[1+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[0+6*i] = random.choice("RPS")
                self.predictors[1+6*i] = random.choice("RPS")
            j=len_size
            #your_his
            while j>=1 and not self.your_his[self.length-j:self.length] in self.your_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.your_his.rfind(self.your_his[self.length-j:self.length],0,self.length-1)
                self.predictors[2+6*i] = self.your_his[j+k]
                self.predictors[3+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[2+6*i] = random.choice("RPS")
                self.predictors[3+6*i] = random.choice("RPS")
            j=len_size
            #my_his
            while j>=1 and not self.my_his[self.length-j:self.length] in self.my_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.my_his.rfind(self.my_his[self.length-j:self.length],0,self.length-1)
                self.predictors[4+6*i] = self.your_his[j+k]
                self.predictors[5+6*i] = beat[self.my_his[j+k]]
            else:
                self.predictors[4+6*i] = random.choice("RPS")
                self.predictors[5+6*i] = random.choice("RPS")

        #Reverse
        for i in range(3):
            temp =""
            search = temp1[(output+input)] #last round
            for start in range(2, min(limit[i],self.length) ):
                if search == self.both_his2[self.length-start]:
                    temp+=self.both_his2[self.length-start+1]
            if(temp==""):
                self.predictors[12+i] = random.choice("RPS")
            else:
                collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
                for sdf in temp:
                    next_move = temp2[sdf]
                    if(who_win[next_move]==-1):
                        collectR[temp2[sdf][1]]+=3
                    elif(who_win[next_move]==0):
                        collectR[temp2[sdf][1]]+=1
                    elif(who_win[next_move]==1):
                        collectR[beat[temp2[sdf][0]]]+=1
                max1 = -1
                p1 =""
                for key in collectR:
                    if(collectR[key]>max1):
                        max1 = collectR[key]
                        p1 += key
                self.predictors[12+i] = random.choice(p1)

        for i in range(15,45):
            self.predictors[i]=beat[beat[self.predictors[i-15]]]
        #find prediction for each meta
        self.metapredictors[0]=self.predictors[self.score1.index(max(self.score1))]
        self.metapredictors[1]=self.predictors[self.score2.index(max(self.score2))]
        self.metapredictors[2]=beat[self.predictors[self.score3.index(max(self.score3))]]
        self.metapredictors[3]=beat[self.predictors[self.score4.index(max(self.score4))]]

        predict = self.metapredictors[self.metascore.index(max(self.metascore))]
        output = beat[predict]
        #output = random.choice(not_lose[predict])
        
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 49 switching20_fix

In [ ]:
%%writefile -a submission.py

# switching20_fix    
# http://www.rpscontest.com/entry/175002
class switching20_fix(agent):
    def __init__(self, mode='normal'):
        
        self.both_hist = ""
        self.my_hist = ""
        self.opp_hist = ""

        
        self.both_patterns = defaultdict(str)
        self.opp_patterns = defaultdict(str)
        self.my_patterns = defaultdict(str)

        self.both2_patterns = defaultdict(str)
        self.opp2_patterns = defaultdict(str)
        self.my2_patterns = defaultdict(str)
        
        self.candidates = []
        self.performance = [None]
        
        self.mode = mode
        
        
    def history_step(self, history):
        beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        score = {'RR': 0, 'PP': 0, 'SS': 0, 'PR': 1, 'RS': 1, 'SP': 1,'RP': -1, 'SR': -1, 'PS': -1,}
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        
        if len(history) == 1:
            self.candidates = [output] * 36
            self.performance = [(0,0)] * 36
            
        for length in range(min(5, len(self.my_hist)), 0, -1):
            p = self.opp_patterns[self.opp_hist[-length:]]
            if p != "":
                for length2 in range(min(5, len(p)), 0, -1):
                    self.opp2_patterns[p[-2*length2:]] += output + input
            self.opp_patterns[self.opp_hist[-length:]] += output + input

            p = self.my_patterns[self.my_hist[-length:]]
            if p != "":
                for length2 in range(min(5, len(p)), 0, -1):
                    self.my2_patterns[p[-2*length2:]] += output + input
            self.my_patterns[self.my_hist[-length:]] += output + input

            p = self.both_patterns[self.both_hist[-2*length:]]
            if p != "":
                for length2 in range(min(5, len(p)), 0, -1):
                    self.both2_patterns[p[-2*length2:]] += output + input
            self.both_patterns[self.both_hist[-2*length:]] += output + input
        
        self.both_hist += output+input
        self.my_hist += output
        self.opp_hist += input

        for i, c in enumerate(self.candidates):
            self.performance[i] = ({1:self.performance[i][0]+1, 0: 0, -1: 0}[score[c+input]],  
                        self.performance[i][1]+score[c+input])
            
        output = random.choice(['R', 'P', 'S'])
        self.candidates = [output] * 36

        idx = self.performance.index(max(self.performance, key=lambda x: x[0]**3+x[1]))

        
        for length in range(min(5, len(self.my_hist)), 0, -1):
            pattern = self.both_patterns[self.both_hist[-2*length:]]
            if pattern != "":
                opp = pattern[-1]
                my = pattern[-2]
                self.candidates[0] = beat[opp]
                self.candidates[1] = cede[my]
                self.candidates[2] = opp
                self.candidates[3] = my
                self.candidates[4] = cede[opp]
                self.candidates[5] = beat[my]
                for length2 in range(min(5, len(pattern)), 0, -1):
                    pattern2 = self.both2_patterns[pattern[-2*length2:]]
                    if pattern2 != "":
                        my2 = pattern2[-2]
                        opp2 = pattern2[-1]
                        self.candidates[6] = beat[opp2]
                        self.candidates[7] = cede[my2]
                        self.candidates[8] = opp2
                        self.candidates[9] = my2
                        self.candidates[10] = cede[opp2]
                        self.candidates[11] = beat[my2]
                        break
                break
                    
        for length in range(min(5, len(self.my_hist)), 0, -1):
            pattern = self.my_patterns[self.my_hist[-length:]]
            if pattern != "":
                opp = pattern[-1]
                my = pattern[-2]
                self.candidates[24] = beat[opp]
                self.candidates[25] = cede[my]
                self.candidates[26] = opp
                self.candidates[27] = my
                self.candidates[28] = cede[opp]
                self.candidates[29] = beat[my]
                for length2 in range(min(5, len(pattern)), 0, -1):
                    pattern2 = self.my2_patterns[pattern[-2*length2:]]
                    if pattern2 != "":
                        my2 = pattern2[-2]
                        opp2 = pattern2[-1]
                        self.candidates[30] = beat[opp2]
                        self.candidates[31] = cede[my2]
                        self.candidates[32] = opp2
                        self.candidates[33] = my2
                        self.candidates[34] = cede[opp2]
                        self.candidates[35] = beat[my2]
                        break
                break
                
        for length in range(min(5, len(self.opp_hist)), 0, -1):
            pattern = self.opp_patterns[self.opp_hist[-length:]]
            if pattern != "":
                opp = pattern[-1]
                my = pattern[-2]
                self.candidates[12] = beat[opp]
                self.candidates[13] = cede[my]
                self.candidates[14] = opp
                self.candidates[15] = my
                self.candidates[16] = cede[opp]
                self.candidates[17] = beat[my]
                for length2 in range(min(5, len(pattern)), 0, -1):
                    pattern2 = self.opp2_patterns[pattern[-2*length2:]]
                    if pattern2 != "":
                        my2 = pattern2[-2]
                        opp2 = pattern2[-1]
                        self.candidates[18] = beat[opp2]
                        self.candidates[19] = cede[my2]
                        self.candidates[20] = opp2
                        self.candidates[21] = my2
                        self.candidates[22] = cede[opp2]
                        self.candidates[23] = beat[my2]
                        break
                break
                
        output = self.candidates[idx]

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## rank 113 DNA werfer 5

In [ ]:
%%writefile -a submission.py
    
    
# DNA_werfer5  
# http://www.rpscontest.com/entry/112004
class DNA_werfer5(agent):
    def __init__(self, mode='normal'):
        
        number_of_predictors = 36 #yes, this really has 36 predictors.
        self.predictors=['' for i in range(number_of_predictors)]
        for i in range(number_of_predictors):
            self.predictors[i]=random.choice(['R','P','S'])
        
        self.urmoves=""
        self.mymoves=""
        self.DNAmoves=""
        
        self.predictorscore=[0.0 for i in range(number_of_predictors)]

        self.length=0
        self.oldpredictors = []
        
        self.mode = mode

    def initial_step(self):
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        output=random.choice(['R','P','S'])

        self.oldpredictors = self.predictors
        return rev_dic[output]
        
    def history_step(self, history):
        number_of_predictors = 36 #yes, this really has 36 predictors.
        
        beat={'R':'P','P':'S','S':'R'}
        nuclease={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}

        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
    
        for i in range(number_of_predictors):
            self.predictorscore[i]*=0.8
            self.predictorscore[i]+=(input==self.oldpredictors[i])*3
            self.predictorscore[i]-=(input=={'R':'S', 'P':'R', 'S':'P'}[self.oldpredictors[i]])*3

        #History matching
        self.urmoves+=input
        self.mymoves+=output
        self.DNAmoves+=nuclease[input+output]
        self.length+=1

        limit = min([self.length,10])
        j=limit
        while j>=1 and not self.DNAmoves[self.length-j:self.length] in self.DNAmoves[0:self.length-1]:
            j-=1
        if j>=1:
            i = self.DNAmoves.find(self.DNAmoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[0] = self.urmoves[j+i] 
            self.predictors[1] = beat[self.mymoves[j+i]] 
            i = self.DNAmoves.rfind(self.DNAmoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[2] = self.urmoves[j+i] 
            self.predictors[3] = beat[self.mymoves[j+i]]
            j=limit
        while j>=1 and not self.urmoves[self.length-j:self.length] in self.urmoves[0:self.length-1]:
            j-=1
        if j>=1:
            i = self.urmoves.find(self.urmoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[4] = self.urmoves[j+i] 
            self.predictors[5] = beat[self.mymoves[j+i]] 
            i = self.urmoves.rfind(self.urmoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[6] = self.urmoves[j+i] 
            self.predictors[7] = beat[self.mymoves[j+i]] 
        j=limit
        while j>=1 and not self.mymoves[self.length-j:self.length] in self.mymoves[0:self.length-1]:
            j-=1
        if j>=1:
            i = self.mymoves.find(self.mymoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[8] = self.urmoves[j+i] 
            self.predictors[9] = beat[self.mymoves[j+i]]
            i = self.mymoves.rfind(self.mymoves[self.length-j:self.length],0,self.length-1) 
            self.predictors[10] = self.urmoves[j+i] 
            self.predictors[11] = beat[self.mymoves[j+i]]

        for i in range(12,36):
            self.predictors[i]=beat[beat[self.predictors[i-12]]] #Trying to second guess me?

        #compare predictors
        output = beat[self.predictors[self.predictorscore.index(max(self.predictorscore))]]
        self.oldpredictors = self.predictors
        
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## bayes15

In [ ]:
%%writefile -a submission.py    
    
# bayes15    
# http://www.rpscontest.com/entry/201003
class bayes15(agent):
    def __init__(self, mode='normal'):
        
        self.played_probs = defaultdict(lambda: 1)
        self.opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.my_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.both_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.singleopp_opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singleopp_my_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singleopp_both_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.singlemy_opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singlemy_my_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singlemy_both_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.singleopp_opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singleopp_my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singleopp_both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.singlemy_opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singlemy_my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
        self.singlemy_both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

        self.win_probs = defaultdict(lambda: 1)
        self.lose_probs = defaultdict(lambda: 1)
        self.tie_probs = defaultdict(lambda: 1)

        self.singleopp_win_probs = defaultdict(lambda: 1)
        self.singleopp_lose_probs = defaultdict(lambda: 1)
        self.singleopp_tie_probs = defaultdict(lambda: 1)

        self.singlemy_win_probs = defaultdict(lambda: 1)
        self.singlemy_lose_probs = defaultdict(lambda: 1)
        self.singlemy_tie_probs = defaultdict(lambda: 1)

        self.opp_answers = {'c': 1, 'b': 1, 'r': 1}
        self.my_answers = {'c': 1, 'b': 1, 'r': 1}

        self.opp2_answers = {'c': 1, 'b': 1, 'r': 1}
        self.my2_answers = {'c': 1, 'b': 1, 'r': 1}  

        self.singleopp_opp_answers = {'c': 1, 'b': 1, 'r': 1}
        self.singleopp_my_answers = {'c': 1, 'b': 1, 'r': 1}

        self.singleopp_opp2_answers = {'c': 1, 'b': 1, 'r': 1}
        self.singleopp_my2_answers = {'c': 1, 'b': 1, 'r': 1}  

        self.singlemy_opp_answers = {'c': 1, 'b': 1, 'r': 1}
        self.singlemy_my_answers = {'c': 1, 'b': 1, 'r': 1}

        self.singlemy_opp2_answers = {'c': 1, 'b': 1, 'r': 1}
        self.singlemy_my2_answers = {'c': 1, 'b': 1, 'r': 1}

        self.patterndict = defaultdict(str)
        self.patterndict2 = defaultdict(str)
        self.opppatterndict = defaultdict(str)
        self.opppatterndict2 = defaultdict(str)
        self.mypatterndict = defaultdict(str)
        self.mypatterndict2 = defaultdict(str)

        self.csu = [0] * 6 # consecutive strategy usage
        self.csc = []  # consecutive strategy candidates
        self.singleopp_csu = [0] * 6 # consecutive strategy usage
        self.singleopp_csc = []  # consecutive strategy candidates
        self.singlemy_csu = [0] * 6 # consecutive strategy usage
        self.singlemy_csc = []  # consecutive strategy candidates

        self.hist = "" 
        self.myhist = "" 
        self.opphist = "" 

        self.my = self.opp = self.my2 = self.opp2 =  ""
        self.singleopp_my = self.singleopp_opp = self.singleopp_my2 = self.singleopp_opp2 = ""
        self.singlemy_my = self.singlemy_opp = self.singlemy_my2 = self.singlemy_opp2 = ""

        self.sc = 0
        self.opp_strats = []
        self.singleopp_oppstrats = []
        self.singlemy_oppstrats = []
        self.addval = 1.0
        
        self.mode = mode
        
    def history_step(self, history):
        
        
        def counter_prob(probs):
            weighted_list = []
            for h in ['R', 'P', 'S']:
                weighted = 0
                for p in probs.keys():
                    points = score[h+p]
                    prob = probs[p]
                    weighted += points * prob
                weighted_list.append((h, weighted))

            return max(weighted_list, key=operator.itemgetter(1))[0]
        
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}

        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
        score = {'RR': 0, 'PP': 0, 'SS': 0, 'PR': 1, 'RS': 1, 'SP': 1,'RP': -1, 'SR': -1, 'PS': -1,}
        cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', 'PR': 'b', 'RS': 'b', 'SP': 'b','RP': 'c', 'SR': 'c', 'PS': 'c',}
        beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        rps = ['R', 'P', 'S']
              
        self.addval += 0.1
        previous_opp_strats = self.opp_strats[:]
        previous_singleopp_oppstrats = self.singleopp_oppstrats[:]
        previous_singlemy_oppstrats = self.singlemy_oppstrats[:]
        previous_sc = self.sc

        self.sc = score[output + input]
        for i, c in enumerate(self.csc):
            if c == input:
                self.csu[i] += 1
            else:
                self.csu[i] = 0

        for i, c in enumerate(self.singleopp_csc):
            if c == input:
                self.singleopp_csu[i] += 1
            else:
                self.singleopp_csu[i] = 0

        for i, c in enumerate(self.singlemy_csc):
            if c == input:
                self.singlemy_csu[i] += 1
            else:
                self.singlemy_csu[i] = 0

        m = max(self.csu)
        self.opp_strats = [i for i, c in enumerate(self.csc) if self.csu[i] == m]

        m = max(self.singleopp_csu)
        self.singleopp_oppstrats = [i for i, c in enumerate(self.singleopp_csc) if self.singleopp_csu[i] == m]

        m = max(self.csu)
        self.singlemy_oppstrats = [i for i, c in enumerate(self.singlemy_csc) if self.singlemy_csu[i] == m]

        if previous_sc == 1:
            for s1 in previous_opp_strats:
                for s2 in self.opp_strats:
                    self.win_probs[chr(s1)+chr(s2)] += self.addval

            for s1 in previous_singleopp_oppstrats:
                for s2 in self.singleopp_oppstrats:
                    self.singleopp_win_probs[chr(s1)+chr(s2)] += self.addval

            for s1 in previous_singlemy_oppstrats:
                for s2 in singlemy_oppstrats:
                    self.singlemy_win_probs[chr(s1)+chr(s2)] += self.addval

        if previous_sc == 0:
            for s1 in previous_opp_strats:
                for s2 in self.opp_strats:
                    self.tie_probs[chr(s1)+chr(s2)] += self.addval

            for s1 in previous_singleopp_oppstrats:
                for s2 in self.singleopp_oppstrats:
                    self.singleopp_tie_probs[chr(s1)+chr(s2)] += self.addval

            for s1 in previous_singlemy_oppstrats:
                for s2 in self.singlemy_oppstrats:
                    self.singlemy_tie_probs[chr(s1)+chr(s2)] += self.addval

        if previous_sc == -1:
            for s1 in previous_opp_strats:
                for s2 in self.opp_strats:
                    self.lose_probs[chr(s1)+chr(s2)] += self.addval
            for s1 in previous_singleopp_oppstrats:
                for s2 in self.singleopp_oppstrats:
                    self.singleopp_lose_probs[chr(s1)+chr(s2)] += self.addval
            for s1 in previous_singlemy_oppstrats:
                for s2 in self.singlemy_oppstrats:
                    self.singlemy_lose_probs[chr(s1)+chr(s2)] += self.addval

        if self.my and self.opp:
            self.opp_answers[cscore[input+self.opp]] += self.addval
            self.my_answers[cscore[input+self.my]] += self.addval
        if self.my2 and self.opp2:
            self.opp2_answers[cscore[input+self.opp2]] += self.addval
            self.my2_answers[cscore[input+self.my2]] += self.addval

        if self.singleopp_my and self.singleopp_opp:
            self.singleopp_opp_answers[cscore[input+self.singleopp_opp]] += self.addval
            self.singleopp_my_answers[cscore[input+self.singleopp_my]] += self.addval
        if self.singleopp_my2 and self.singleopp_opp2:
            self.singleopp_opp2_answers[cscore[input+self.singleopp_opp2]] += self.addval
            self.singleopp_my2_answers[cscore[input+self.singleopp_my2]] += self.addval

        if self.singlemy_my and self.singlemy_opp:
            self.singlemy_opp_answers[cscore[input+self.singlemy_opp]] += self.addval
            self.singlemy_my_answers[cscore[input+self.singlemy_my]] += self.addval
        if self.singlemy_my2 and self.singlemy_opp2:
            self.singlemy_opp2_answers[cscore[input+self.singlemy_opp2]] += self.addval
            self.singlemy_my2_answers[cscore[input+self.singlemy_my2]] += self.addval

        for length in range(min(10, len(self.hist)), 0, -2):
            pattern = self.patterndict[self.hist[-length:]]
            if pattern:
                for length2 in range(min(10, len(pattern)), 0, -2):
                    self.patterndict2[pattern[-length2:]] += output + input
            self.patterndict[self.hist[-length:]] += output + input

        # singleopp
        for length in range(min(5, len(self.opphist)), 0, -1):
            pattern = self.opppatterndict[self.opphist[-length:]]
            if pattern:
                for length2 in range(min(10, len(pattern)), 0, -2):
                    self.opppatterndict2[pattern[-length2:]] += output + input
            self.opppatterndict[self.opphist[-length:]] += output + input

        # singlemy
        for length in range(min(5, len(self.myhist)), 0, -1):
            pattern = self.mypatterndict[self.myhist[-length:]]
            if pattern:
                for length2 in range(min(10, len(pattern)), 0, -2):
                    self.mypatterndict2[pattern[-length2:]] += output + input
            self.mypatterndict[self.myhist[-length:]] += output + input

        self.played_probs[input] += self.addval
        self.opp_probs[self.opp][input] += self.addval
        self.my_probs[self.my][input] += self.addval
        self.both_probs[self.my+self.opp][input] += self.addval

        self.opp2_probs[self.opp2][input] += self.addval
        self.my2_probs[self.my2][input] += self.addval
        self.both2_probs[self.my2+self.opp2][input] += self.addval

        self.hist += output + input
        self.myhist += output
        self.opphist += input

        self.my = self.opp = self.my2 = self.opp2 = ""
        self.singleopp_my = self.singleopp_opp = self.singleopp_my2 = self.singleopp_opp2 = ""
        self.singlemy_my = self.singlemy_opp = self.singlemy_my2 = self.singlemy_opp2 = ""

        for length in range(min(10, len(self.hist)), 0, -2):
            pattern = self.patterndict[self.hist[-length:]]
            if pattern != "":
                self.my = pattern[-2]
                self.opp = pattern[-1]
                for length2 in range(min(10, len(pattern)), 0, -2):
                    pattern2 = self.patterndict2[pattern[-length2:]]
                    if pattern2 != "":
                        self.my2 = pattern2[-2]
                        self.opp2 = pattern2[-1]
                        break
                break

        # singleopp
        for length in range(min(5, len(self.opphist)), 0, -1):
            pattern = self.opppatterndict[self.opphist[-length:]]
            if pattern != "":
                self.singleopp_my = pattern[-2]
                self.singleopp_opp = pattern[-1]
                for length2 in range(min(10, len(pattern)), 0, -2):
                    pattern2 = self.opppatterndict2[pattern[-length2:]]
                    if pattern2 != "":
                        self.singleopp_my2 = pattern2[-2]
                        self.singleopp_opp2 = pattern2[-1]
                        break
                break

        # singlemy
        for length in range(min(5, len(self.myhist)), 0, -1):
            pattern = self.mypatterndict[self.myhist[-length:]]
            if pattern != "":
                self.singlemy_my = pattern[-2]
                self.singlemy_opp = pattern[-1]
                for length2 in range(min(10, len(pattern)), 0, -2):
                    pattern2 = self.mypatterndict2[pattern[-length2:]]
                    if pattern2 != "":
                        self.singlemy_my2 = pattern2[-2]
                        self.singlemy_opp2 = pattern2[-1]
                        break
                break

        probs = {}
        for hand in rps:
            probs[hand] = self.played_probs[hand]

        if self.my and self.opp:
            for hand in rps:
                probs[hand] *= self.opp_probs[self.opp][hand] * self.my_probs[self.my][hand] * self.both_probs[self.my+self.opp][hand]
                probs[hand] *= self.opp_answers[cscore[hand+self.opp]] * self.my_answers[cscore[hand+self.my]]
        
            csc = [self.opp, beat[self.opp], cede[self.opp], self.my, cede[self.my], beat[self.my]]


            strats_for_hand = {'R': [], 'P': [], 'S': []}
            for i, c in enumerate(csc):
                strats_for_hand[c].append(i)

            if self.sc == 1:
                pr = self.win_probs
            if self.sc == 0:
                pr = self.tie_probs
            if self.sc == -1:
                pr = self.lose_probs

            for hand in rps:
                for s1 in self.opp_strats:
                    for s2 in strats_for_hand[hand]:
                        probs[hand] *= pr[chr(s1)+chr(s2)]
        else:
            csc = []

        if self.singleopp_my and self.singleopp_opp:
            for hand in rps:
                probs[hand] *= self.singleopp_opp_probs[self.singleopp_opp][hand] * \
                             self.singleopp_my_probs[self.singleopp_my][hand] * \
                             self.singleopp_both_probs[self.singleopp_my+self.singleopp_opp][hand]
                probs[hand] *= self.singleopp_opp_answers[cscore[hand+self.singleopp_opp]] * self.singleopp_my_answers[cscore[hand+self.singleopp_my]]

            singleopp_csc = [self.singleopp_opp, beat[self.singleopp_opp], cede[self.singleopp_opp], self.singleopp_my, cede[self.singleopp_my], beat[self.singleopp_my]]

            strats_for_hand = {'R': [], 'P': [], 'S': []}
            for i, c in enumerate(singleopp_csc):
                strats_for_hand[c].append(i)

            if self.sc == 1:
                pr = self.singleopp_win_probs
            if self.sc == 0:
                pr = self.singleopp_tie_probs
            if self.sc == -1:
                pr = self.singleopp_lose_probs

            for hand in rps:
                for s1 in self.singleopp_oppstrats:
                    for s2 in strats_for_hand[hand]:
                        probs[hand] *= pr[chr(s1)+chr(s2)]
        else:
            singleopp_csc = []

        if self.singlemy_my and self.singlemy_opp:
            for hand in rps:
                probs[hand] *= self.singlemy_opp_probs[self.singlemy_opp][hand] * \
                             self.singlemy_my_probs[self.singlemy_my][hand] * \
                             self.singlemy_both_probs[self.singlemy_my+self.singlemy_opp][hand]
                probs[hand] *= self.singlemy_opp_answers[cscore[hand+self.singlemy_opp]] * self.singlemy_my_answers[cscore[hand+self.singlemy_my]]

            singlemy_csc = [self.singlemy_opp, beat[self.singlemy_opp], cede[self.singlemy_opp], self.singlemy_my, cede[self.singlemy_my], beat[self.singlemy_my]]

            strats_for_hand = {'R': [], 'P': [], 'S': []}
            for i, c in enumerate(singlemy_csc):
                strats_for_hand[c].append(i)

            if self.sc == 1:
                pr = self.singlemy_win_probs
            if self.sc == 0:
                pr = self.singlemy_tie_probs
            if self.sc == -1:
                pr = self.singlemy_lose_probs

            for hand in rps:
                for s1 in self.singlemy_oppstrats:
                    for s2 in strats_for_hand[hand]:
                        probs[hand] *= pr[chr(s1)+chr(s2)]
        else:
            singlemy_csc = []

        if self.my2 and self.opp2:
            for hand in rps:
                probs[hand] *= self.opp2_probs[self.opp2][hand] * self.my2_probs[self.my2][hand] * self.both2_probs[self.my2+self.opp2][hand]
                probs[hand] *= self.opp2_answers[cscore[hand+self.opp2]] * self.my2_answers[cscore[hand+self.my2]]

        if self.singleopp_my2 and self.singleopp_opp2:
            for hand in rps:
                probs[hand] *= self.singleopp_opp2_probs[self.singleopp_opp2][hand] *\
                             self.singleopp_my2_probs[self.singleopp_my2][hand] *\
                             self.singleopp_both2_probs[self.singleopp_my2+self.singleopp_opp2][hand]
                probs[hand] *= self.singleopp_opp2_answers[cscore[hand+self.singleopp_opp2]] * \
                             self.singleopp_my2_answers[cscore[hand+self.singleopp_my2]]

        if self.singlemy_my2 and self.singlemy_opp2:
            for hand in rps:
                probs[hand] *= self.singlemy_opp2_probs[self.singlemy_opp2][hand] *\
                             self.singlemy_my2_probs[self.singlemy_my2][hand] *\
                             self.singlemy_both2_probs[self.singlemy_my2+self.singlemy_opp2][hand]
                probs[hand] *= self.singlemy_opp2_answers[cscore[hand+self.singlemy_opp2]] * \
                             self.singlemy_my2_answers[cscore[hand+self.singlemy_my2]]

        output = counter_prob(probs)

        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## disqualified Centrifugal Bumblepuppy

In [ ]:
%%writefile -a submission.py

class Bumblepuppy5(agent):
    def __init__(self, mode='normal'):
        self.mode = mode
        number_of_predictors = 60 #yes, this really has 60 predictors.
        number_of_metapredictors = 12 #actually, I lied! This has 240 predictors.
        
        self.urmoves=""
        self.mymoves=""
        self.DNAmoves=""
        self.outputs=[random.choice("RPS")]*number_of_metapredictors
        self.predictorscore1=[3]*number_of_predictors
        self.predictorscore2=[3]*number_of_predictors
        self.predictorscore3=[3]*number_of_predictors
        self.predictorscore4=[3]*number_of_predictors
        self.nuclease={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
        self.length=0
        self.predictors=[random.choice("RPS")]*number_of_predictors
        self.metapredictors=[random.choice("RPS")]*number_of_metapredictors
        self.metapredictorscore=[3]*number_of_metapredictors
    
    def initial_step(self):
        beat={'R':'P','P':'S','S':'R'}
        
        #compare predictors
        output = beat[self.metapredictors[self.metapredictorscore.index(max(self.metapredictorscore))]]
        if max(self.metapredictorscore)<0:
            output = beat[random.choice(self.urmoves)]

        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
    
    def history_step(self, history):
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        beat={'R':'P','P':'S','S':'R'}
        self.limits = [50,20,6]
        
        number_of_predictors = 60 #yes, this really has 60 predictors.
        number_of_metapredictors = 12 #actually, I lied! This has 240 predictors.
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
            
        for i in range(number_of_predictors):
                #metapredictor 1
            self.predictorscore1[i]*=0.8
            self.predictorscore1[i]+=(input==self.predictors[i])*3
            self.predictorscore1[i]-=(input==beat[beat[self.predictors[i]]])*3
            #metapredictor 2: beat metapredictor 1 (probably contains a bug)
            self.predictorscore2[i]*=0.8
            self.predictorscore2[i]+=(output==self.predictors[i])*3
            self.predictorscore2[i]-=(output==beat[beat[self.predictors[i]]])*3
            #metapredictor 3
            self.predictorscore3[i]+=(input==self.predictors[i])*3
            if input==beat[beat[self.predictors[i]]]:
                self.predictorscore3[i]=0
            #metapredictor 4: beat metapredictor 3 (probably contains a bug)
            self.predictorscore4[i]+=(output==self.predictors[i])*3
            if output==beat[beat[self.predictors[i]]]:
                self.predictorscore4[i]=0

        for i in range(number_of_metapredictors):
            self.metapredictorscore[i]*=0.959
            self.metapredictorscore[i]+=(input==self.metapredictors[i])*3
            self.metapredictorscore[i]-=(input==beat[beat[self.metapredictors[i]]])*3


        #Predictors 1-18: History matching
        self.urmoves+=input
        self.mymoves+=output
        self.DNAmoves+=self.nuclease[input+output]
        self.length+=1

        for z in range(3):
            limit = min([self.length,self.limits[z]])
            j=limit
            while j>=1 and not self.DNAmoves[self.length-j:self.length] in self.DNAmoves[0:self.length-1]:
                j-=1
            if j>=1:
                i = self.DNAmoves.rfind(self.DNAmoves[self.length-j:self.length],0,self.length-1) 
                self.predictors[0+6*z] = self.urmoves[j+i] 
                self.predictors[1+6*z] = beat[self.mymoves[j+i]] 
            j=limit
            while j>=1 and not self.urmoves[self.length-j:self.length] in self.urmoves[0:self.length-1]:
                j-=1
            if j>=1:
                i = self.urmoves.rfind(self.urmoves[self.length-j:self.length],0,self.length-1) 
                self.predictors[2+6*z] = self.urmoves[j+i] 
                self.predictors[3+6*z] = beat[self.mymoves[j+i]] 
            j=limit
            while j>=1 and not self.mymoves[self.length-j:self.length] in self.mymoves[0:self.length-1]:
                j-=1
            if j>=1:
                i = self.mymoves.rfind(self.mymoves[self.length-j:self.length],0,self.length-1) 
                self.predictors[4+6*z] = self.urmoves[j+i] 
                self.predictors[5+6*z] = beat[self.mymoves[j+i]]
        #Predictor 19,20: RNA Polymerase
        L=len(self.mymoves)
        i=self.DNAmoves.rfind(self.DNAmoves[L-j:L-1],0,L-2)
        while i==-1:
            j-=1
            i=self.DNAmoves.rfind(self.DNAmoves[L-j:L-1],0,L-2)
            if j<2:
                break
        if i==-1 or j+i>=L:
            self.predictors[18]=self.predictors[19]=beat[random.choice(self.urmoves)]
        else:
            self.predictors[18]=beat[self.mymoves[j+i]]
            self.predictors[19]=self.urmoves[j+i]

        #Predictors 21-60: rotations of Predictors 1:20
        for i in range(20,60):
            self.predictors[i]=beat[beat[self.predictors[i-20]]] #Trying to second guess me?

        self.metapredictors[0]=self.predictors[self.predictorscore1.index(max(self.predictorscore1))]
        self.metapredictors[1]=beat[self.predictors[self.predictorscore2.index(max(self.predictorscore2))]]
        self.metapredictors[2]=self.predictors[self.predictorscore3.index(max(self.predictorscore3))]
        self.metapredictors[3]=beat[self.predictors[self.predictorscore4.index(max(self.predictorscore4))]]
        for i in range(4,12):
            self.metapredictors[i]=beat[self.metapredictors[i-4]]


        #compare predictors
        output = beat[self.metapredictors[self.metapredictorscore.index(max(self.metapredictorscore))]]
        if max(self.metapredictorscore)<0:
            output = beat[random.choice(self.urmoves)]

    
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

## sample class

In [ ]:
%%writefile -a submission.py

class AgentName(agent):
    def __init__(self, mode='normal'):
        self.mode = mode
    
    def initial_step(self):
    
        rev_dic = {'R': 0, 'P':1, 'S':2}
        return rev_dic[output]
    
    def history_step(self, history):
    
        dic = {0:'R', 1:'P', 2:'S'}
        rev_dic = {'R': 0, 'P':1, 'S':2}
        
        if self.mode == 'normal':
            # normal model
            input = dic[history[-1]['competitorStep']]
            output = dic[history[-1]['step']]
            
        elif self.mode == 'anti':
            # anti model
            input = dic[history[-1]['step']]
            output = dic[history[-1]['competitorStep']]
        
    
        if self.mode == 'normal':
            return rev_dic[output]
        elif self.mode == 'anti':
            return (rev_dic[output] + 1) % 3

# Define Multi-Armed-Bandit Agent

In [ ]:
%%writefile -a submission.py
        
agents = {
    'random': agent(), 
    'anti_random_cracker': anti_random_cracker(), 
    'mirror_0': mirror_shift(0),
    'mirror_1': mirror_shift(1),  
    'mirror_2': mirror_shift(2),
    'self_0': self_shift(0),
    'self_1': self_shift(1),  
    'self_2': self_shift(2),
    'popular_beater': popular_beater(),
    'anti_popular_beater': anti_popular_beater(),
    
    'IO2': IO2(), 
    'dllu1': dllu1(), 
    'RPS_Meta_Fix': RPS_Meta_Fix(),
    'meta_sgd_markov_full_ew': meta_sgd_markov_full_ew(), 
    'AreYouALucker': AreYouALucker(), 
    'RPS_FSa': RPS_FSa(), 
    'SkipTree6/6': SkipTree6_6(), 
    'UCB10': UCB10(), 
    'Testing_Please_Ignore': Testing_Please_Ignore(), 
    'mybot': mybot(), 
    'TheIlliad': TheIlliad(), 
    'RV2_fixing': RV2_fixing(), 
    'switching20_fix': switching20_fix(), 
    'DNA_werfer5': DNA_werfer5(), 
    'bayes15': bayes15(), 
    'CentrifugalBumblepuppy5': Bumblepuppy5(), 
    
    'anti IO2': IO2('anti'), 
    'anti dllu1': dllu1('anti'), 
    'anti RPS_Meta_Fix': RPS_Meta_Fix('anti'),
    'anti meta_sgd_markov_full_ew': meta_sgd_markov_full_ew('anti'), 
    'anti AreYouALucker': AreYouALucker('anti'), 
    'anti RPS_FSa': RPS_FSa('anti'), 
    'anti SkipTree6/6': SkipTree6_6('anti'), 
    'anti UCB10': UCB10('anti'), 
    'anti Testing_Please_Ignore': Testing_Please_Ignore('anti'), 
    'anti mybot': mybot('anti'), 
    'TheIlliad': TheIlliad('anti'), 
    'anti RV2_fixing': RV2_fixing('anti'), 
    'anti switching20_fix': switching20_fix('anti'), 
    'anti DNA_werfer5': DNA_werfer5('anti'), 
    'anti bayes15': bayes15('anti'), 
    'anti CentrifugalBumblepuppy5': Bumblepuppy5('anti'), 
    

    
}


    

history = []
bandit_state = {k:[1,1] for k in agents.keys()}
    
def multi_armed_bandit_agent (observation, configuration):
    
    # bandits' params
    step_size = 4 # how much we increase a and b 
    decay_rate = 1.05 # how much do we decay old historical data
    
    global history, bandit_state
    
    def log_step(step = None, history = None, agent = None, competitorStep = None, file = 'history.csv'):
        if step is None:
            step = np.random.randint(3)
        if history is None:
            history = []
        history.append({'step': step, 'competitorStep': competitorStep, 'agent': agent})
        #if file is not None:
        #    pd.DataFrame(history).to_csv(file, index = False)
        return step
    
    def update_competitor_step(history, competitorStep):
        history[-1]['competitorStep'] = int(competitorStep)
        return history
    
    # load history
    if observation.step == 0:
        pass
    else:
        history = update_competitor_step(history, observation.lastOpponentAction)
        
        # updating bandit_state using the result of the previous step
        # we can update all states even those that were not used
        for name, agent in agents.items():
            agent_step = agent.step(history[:-1])
            bandit_state[name][1] = (bandit_state[name][1] - 1) / decay_rate + 1
            bandit_state[name][0] = (bandit_state[name][0] - 1) / decay_rate + 1
            
            if (history[-1]['competitorStep'] - agent_step) % 3 == 1:
                bandit_state[name][1] += step_size
            elif (history[-1]['competitorStep'] - agent_step) % 3 == 2:
                bandit_state[name][0] += step_size
            else:
                bandit_state[name][0] += step_size/2
                bandit_state[name][1] += step_size/2
            
    # we can use it for analysis later
    #with open('bandit.json', 'w') as outfile:
    #    json.dump(bandit_state, outfile)
    
    
    # generate random number from Beta distribution for each agent and select the most lucky one
    best_proba = -1
    best_agent = None
    for k in bandit_state.keys():
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k
        
    step = agents[best_agent].step(history)
    
    return log_step(step, history, best_agent)

In [ ]:
import json
import pandas as pd
import numpy as np

from kaggle_environments import evaluate, make, utils
env = make("rps", debug=True)

### vs. pattern matching

In [ ]:
%%writefile pattern_matching.py

import random


# current memory of the agent
current_memory = []
# maximum steps in the pattern
steps_max = 6
# minimum steps in the pattern
steps_min = 3
# memory length of patterns in first group
# steps_max is multiplied by 2 to consider both my_agent's and opponent's actions
group_memory_length = steps_max * 2
# list of groups of memory patterns
groups_of_memory_patterns = []
for i in range(steps_max, steps_min - 1, -1):
    groups_of_memory_patterns.append({
        # how many steps in a row are in the pattern
        "memory_length": group_memory_length,
        # list of memory patterns
        "memory_patterns": []
    })
    group_memory_length -= 2

    
def find_pattern(memory_patterns, memory, memory_length):
    """ find appropriate pattern in memory """
    for pattern in memory_patterns:
        actions_matched = 0
        for i in range(memory_length):
            if pattern["actions"][i] == memory[i]:
                actions_matched += 1
            else:
                break
        # if memory fits this pattern
        if actions_matched == memory_length:
            return pattern
    # appropriate pattern not found
    return None

def my_agent(obs, conf):
    """ your ad here """
    # action of my_agent
    my_action = None
    # if it's not first step, add opponent's last action to agent's current memory
    if obs["step"] > 0:
        current_memory.append(obs["lastOpponentAction"])
    for group in groups_of_memory_patterns:
        # if length of current memory is bigger than necessary for a new memory pattern
        if len(current_memory) > group["memory_length"]:
            # get momory of the previous step
            previous_step_memory = current_memory[:group["memory_length"]]
            previous_pattern = find_pattern(group["memory_patterns"], previous_step_memory, group["memory_length"])
            if previous_pattern == None:
                previous_pattern = {
                    "actions": previous_step_memory.copy(),
                    "opp_next_actions": [
                        {"action": 0, "amount": 0, "response": 1},
                        {"action": 1, "amount": 0, "response": 2},
                        {"action": 2, "amount": 0, "response": 0}
                    ]
                }
                group["memory_patterns"].append(previous_pattern)
            for action in previous_pattern["opp_next_actions"]:
                if action["action"] == obs["lastOpponentAction"]:
                    action["amount"] += 1
            # delete first two elements in current memory (actions of the oldest step in current memory)
            del current_memory[:2]
            # if action was not yet found
            if my_action == None:
                pattern = find_pattern(group["memory_patterns"], current_memory, group["memory_length"])
                # if appropriate pattern is found
                if pattern != None:
                    my_action_amount = 0
                    for action in pattern["opp_next_actions"]:
                        # if this opponent's action occurred more times than currently chosen action
                        # or, if it occured the same amount of times and this one is choosen randomly among them
                        if (action["amount"] > my_action_amount or
                                (action["amount"] == my_action_amount and random.random() > 0.5)):
                            my_action_amount = action["amount"]
                            my_action = action["response"]
    # if no action is found
    if my_action == None:
        my_action = random.randint(0, 2)
    current_memory.append(my_action)
    return my_action

In [ ]:
env.reset()
env.run(["submission.py", "pattern_matching.py"])
env.render(mode="ipython", width=500, height=500)

In [ ]:
with open('bandit.json') as json_file:
    bandit_state = json.load(json_file)
bandit_state

In [ ]:
pd.read_csv('history.csv')['agent'].value_counts()